In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_df = pd.read_csv("/content/movies.csv")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings_df = pd.read_csv("/content/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [4]:
ratings_df['userId'].value_counts()

668    5678
575    2837
458    2086
232    1421
310    1287
       ... 
58       20
51       20
288      20
388      20
257      20
Name: userId, Length: 668, dtype: int64

In [5]:
ratings_df = ratings_df.drop(['timestamp'], axis=1)
average_ratings_df = ratings_df.groupby(['movieId']).mean().reset_index()
average_ratings_df.head()


,movieId,userId,rating
0,1,335.663793,3.907328
1,2,338.173913,3.353261
2,3,299.982759,3.189655
3,4,342.727273,2.818182
4,5,284.822581,3.250000


In [6]:
avg_ratings_df = ratings_df.groupby('movieId').agg({'rating': 'mean', 'userId': 'count'}).reset_index()
avg_ratings_df.head()

,movieId,rating,userId
0,1,3.907328,232
1,2,3.353261,92
2,3,3.189655,58
3,4,2.818182,11
4,5,3.250000,62


In [20]:
Final_df = pd.merge(movie_df, avg_ratings_df, how='left', on='movieId')
Final_df.head()


,movieId,title,genres,rating,userId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.907328,232.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.353261,92.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.189655,58.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.818182,11.0
4,5,Father of the Bride Part II (1995),Comedy,3.250000,62.0


In [13]:
movie_df.head()
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(stop_words='english')
tf_matrix = tf.fit_transform(movie_df['genres'])

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tf_matrix)
def recommend_movies(movie_title, n):
  movie_index = movie_df[movie_df['title'] == movie_title].index[0]
  similar_movies = cosine_sim[movie_index]
  top_n_movies = np.argsort(similar_movies)[-n:]
  return movie_df['title'][top_n_movies]
recommend_movies('Toy Story (1995)', 5)

3166                     Emperor's New Groove, The (2000)
9215    Asterix and the Vikings (Astérix et les Viking...
9732                                         Turbo (2013)
2496                                   Toy Story 2 (1999)
0                                        Toy Story (1995)
Name: title, dtype: object

In [15]:
recommend_movies('Jumanji (1995)', 5)

8795    Chronicles of Narnia: The Voyage of the Dawn T...
817                       Escape to Witch Mountain (1975)
1652                                  Return to Oz (1985)
1908                        Santa Claus: The Movie (1985)
1604            Darby O'Gill and the Little People (1959)
Name: title, dtype: object

In [16]:
recommend_movies('Grumpier Old Men (1995)', 5)

8867              No Strings Attached (2011)
5269    Win a Date with Tad Hamilton! (2004)
5267                               It (1927)
3717                Waking Up in Reno (2002)
1306              Smile Like Yours, A (1997)
Name: title, dtype: object

In [18]:
recommend_movies('Waiting to Exhale (1995)', 5)

9693                    Much Ado About Nothing (2012)
10041    Very Ordinary Couple (Yeonaeui Wondo) (2013)
4470                                 My Girl 2 (1994)
4178                                   Pumpkin (2002)
3415                How to Marry a Millionaire (1953)
Name: title, dtype: object

In [19]:
recommend_movies('Father of the Bride Part II (1995)', 5)

3614                             Let It Ride (1989)
2858                           Things Change (1988)
5042                     Father of the Bride (1991)
5033                Solid Gold Cadillac, The (1956)
2154    Dinner Game, The (Dîner de cons, Le) (1998)
Name: title, dtype: object

**Item** **Based** **Collaborative** **Filtering** **bold text**

In [28]:
avg_ratings_df.fillna(value=0, inplace=True)
movies_sim = cosine_similarity(avg_ratings_df)
np.fill_diagonal(movies_sim,0)
movies_sim_df = pd.DataFrame(movies_sim, columns = avg_ratings_df.index, index = avg_ratings_df.index)
movies_sim_df.idxmax(axis=1).head()

0    1
1    0
2    5
3    8
4    9
dtype: int64

In [35]:
import ipywidgets as widgets
import random
# Create a text input widget for the user to enter a movie title
movie_title_input = widgets.Text(
    description="Movie Title:",
    placeholder="Enter a movie title",
    style={"width": "200px"},
)
# Create a slider widget for the user to select the number of recommendations
num_recommendations_slider = widgets.IntSlider(
    description="Number of Recommendations:",
    min=1,
    max=10,
    value=5,
    step=1,
    style={"width": "200px"},
)
# Create a button widget for the user to click to generate recommendations
generate_recommendations_button = widgets.Button(
    description="Generate Recommendations",
    style={"width": "200px"},
)
# Create a text output widget to display the recommendations
recommendations_output = widgets.Textarea(
    description="Recommendations:",
    disabled=True,
    style={"width": "400px", "height": "200px"},
)
# Create a layout for the widgets
layout = widgets.Layout(
    display="flex",
    flex_direction="column",
    align_items="center",
    justify_content="space-between",
)
# Add the widgets to the layout
layout.children = [
    movie_title_input,
    num_recommendations_slider,
    generate_recommendations_button,
    recommendations_output,
]
# Display the layout in the notebook
display(layout)
# Define the function to generate recommendations
def generate_recommendations(movie_title, num_recommendations):
    # Get the index of the movie in the movie dataframe
    movie_index = movie_df[movie_df["title"] == movie_title].index[0]
    # Calculate the similarity between the movie and all other movies
    similar_movies = cosine_sim[movie_index]
    # Sort the movies by similarity in descending order
    sorted_movies = np.argsort(similar_movies)[-num_recommendations:]
    # Get the titles of the recommended movies
    recommended_movies = movie_df["title"][sorted_movies]
    # Return the list of recommended movies
    return recommended_movies
# Define the event handler for the generate_recommendations_button
def on_generate_recommendations_button_clicked(event):
    # Get the movie title from the movie_title_input widget
    movie_title = movie_title_input.value
    # Get the number of recommendations from the num_recommendations_slider widget
    num_recommendations = num_recommendations_slider.value
    # Generate the recommendations
    recommended_movies = generate_recommendations(movie_title, num_recommendations)
    # Update the recommendations_output widget with the list of recommended movies
    recommendations_output.value = "\n".join(recommended_movies)
# Register the event handler for the generate_recommendations_button
generate_recommendations_button.on_click(on_generate_recommendations_button_clicked)

Layout(align_items='center', display='flex', justify_content='space-between')